In [2]:
# import dependencies
import gspread
import pandas as pd
import re
from oauth2client.service_account import ServiceAccountCredentials
from name_scrubber import name_map

In [3]:
# authenticate and connect to google sheets
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("utilities.json", scope)
client = gspread.authorize(creds)
spreadsheet = client.open("Utilities 313")

In [4]:
# function to clean currency with regex
def clean_currency(value):
    if value:
        return float(re.sub(r"[^\d.]", "", value))
    return None

In [5]:
# empty list to extract data
data = []

# loop through each sheet to get data
for sheet in spreadsheet.worksheets():
    sheet_name = sheet.title

    # ignore unwanted sheets
    if sheet_name.lower() in ["template", "pepco update"]:
        continue

    # read values from A2:C6
    values = sheet.get("A2:C6")

    # process rows
    for row in values:
        if len(row) >= 3:
            expense, name, amount = row[0], row[1], clean_currency(row[2])

            # convert sheet name to datetime
            date = pd.to_datetime(sheet_name, errors="coerce")
            if pd.notna(date):
                data.append([date, expense, name, amount])

In [6]:
# save to df
df = pd.DataFrame(data, columns=["Date", "Expense", "Name", "Amount"])

# add pseudonyms
df = name_map(df)

In [7]:
# confirm pseudonyms
df.Name.unique()

array(['Randy Detman', 'Morning Glory', 'Zen Master Shabaka'],
      dtype=object)

In [8]:
# data cleaning
# update Pepco values and move CleanChoice entry

# update Pepco values for Nov 2023 - May 2024
# accounting for data in "Pepco Update" sheet
pepco_updates = {
    "2023-12-01": 192.81,
    "2024-01-01": 196.28,
    "2024-02-01": 174.56,
    "2024-03-01": 158.60,
    "2024-04-01": 139.34,
}

for date, amount in pepco_updates.items():
    df.loc[(df["Date"] == date) & (df["Expense"] == "Pepco"), "Amount"] = amount

# move CleanChoice from March 2023 to February 2023
# legacy from Zen Master Shabaka moving out
mask = (df["Date"] == "2023-03-01") & (df["Expense"] == "CleanChoice")
if mask.any():
    df.loc[mask, "Date"] = "2023-02-01"

# gotta check w/ Detman if this is necessary
# subtract 143.5 from CleanChoice amount for June 2023
# df.loc[(df["Date"] == "2023-06-01") & (df["Expense"] == "CleanChoice"), "Amount"] -= 143.5

In [9]:
# change "Pepco/Exelon" to "Pepco" in the Expense column
df.loc[df['Expense'] == "Pepco/Exelon", 'Expense'] = "Pepco"

In [10]:
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month

In [11]:
# in case it's not sorted
df = df.sort_values(by="Date")

# save to csv
df.to_csv("utilities_313.csv", index=False)